# Explotation Zone
Script on es combinen les diverses bases de dades per obtenir un format adequat i necessari pels models triats en la següent fase.

Requeriments previs per a utilitzar aquest script:

In [22]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit, min, max, mean,udf,regexp_replace
from pyspark.sql.types import StringType
from unidecode import unidecode

In [23]:
# Funció per treure els accents i la L de les paraules
def treure_accents_L(cadena):
    cadena_sense_accents = unidecode(cadena)
    return cadena_sense_accents.replace("L'", "")

In [ ]:
# Sessió Spark
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

### Preparació pel primer pipeline (KNN + KMeans)

Es vol ajuntar per districtes les variables de lloguer per mes, lloguer per m^2 i import de renda. Les variables de lloguer es troben juntes en renda i per poder separar-les s'ha de filtar pel tipus euro/mes per una banda i euro/m2 per l'altre. Per les 3 bases s'obtenir la mitjana per any per tal de tenir una única observació per districtes i any.

In [25]:
# Llegir la taula de rent_price
DF1 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:trusted_zone/freshdata_trusted.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", '''SELECT Year, District, AVG(Price_Escalat) AS Year_Price_month FROM rent_price WHERE "Average _Rent" = 'average rent (euro/month)' GROUP BY Year ,District''') \
  .load()
DF1.show()

+----+-------------------+-------------------+
|Year|           District|   Year_Price_month|
+----+-------------------+-------------------+
|2014|           Eixample|  0.333091261995637|
|2015|             Gracia|0.28993265174502825|
|2016|     Sants-Montjuic| 0.2749316692108045|
|2018|     Horta-Guinardo|0.32171624393389936|
|2018|         Nou_Barris|  0.267811108220087|
|2019|           Eixample|0.49606870861218894|
|2020|           Eixample|  0.487231673063165|
|2021|           Eixample| 0.4536786913786479|
|2022|     Sants-Montjuic| 0.3929317812573789|
|2014|             Gracia|0.26889980229005206|
|2015|           Eixample|0.36026206259757737|
|2016|Sarria-Sant_Gervasi| 0.5516446753997372|
|2016|         Sant_Marti| 0.3478377456836853|
|2017|        Sant_Andreu|0.27965781377203086|
|2018|       Ciutat_Vella| 0.4159131397819904|
|2018|          Les_Corts| 0.5793778127852433|
|2019|             Gracia|  0.414363574849603|
|2020|             Gracia|0.42102174809426635|
|2021|       

In [26]:
# Llegir la taula de compravenda_sup
DF2 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:trusted_zone/freshdata_trusted.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", '''SELECT Year, District, AVG(Price_Escalat) AS Year_Price_m2 FROM rent_price WHERE "Average _Rent" = 'average rent per surface (euro/m2)' GROUP BY Year ,District''') \
  .load()
DF2.show()

+----+-------------------+-------------------+
|Year|           District|      Year_Price_m2|
+----+-------------------+-------------------+
|2014|       Ciutat_Vella| 0.4785458057395143|
|2014|          Les_Corts| 0.4477097130242826|
|2015|     Horta-Guinardo|0.36386151940989603|
|2015|         Nou_Barris| 0.2866209397666351|
|2016|        Sant_Andreu| 0.3477457972491085|
|2017|Sarria-Sant_Gervasi| 0.7045851729212657|
|2017|         Sant_Marti| 0.5433774834437085|
|2018|             Gracia|   0.56401766004415|
|2019|       Ciutat_Vella| 0.6685292494481235|
|2019|          Les_Corts| 0.6541574687270052|
|2020|       Ciutat_Vella| 0.6471440397350993|
|2020|          Les_Corts| 0.6444996320824136|
|2021|       Ciutat_Vella| 0.5847130242825607|
|2021|          Les_Corts| 0.5975901398086828|
|2022|        Sant_Andreu|0.47065857247976456|
|2014|Sarria-Sant_Gervasi| 0.4780629139072848|
|2014|         Sant_Marti|0.37310982339955845|
|2015|     Sants-Montjuic|0.38754598969830756|
|2016|       

In [27]:
# Llegir la taula de renda
DF3 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:trusted_zone/freshdata_trusted.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT \"Any\", Nom_Districte, AVG(Import_Euros_Escalat) AS District_Income FROM renda GROUP BY \"Any\",Nom_Districte") \
  .load()
DF3.show()

+----+-------------------+-------------------+
| Any|      Nom_Districte|    District_Income|
+----+-------------------+-------------------+
|2018|       Ciutat_Vella|0.11546314814814815|
|2018|     Sants-Montjuïc|0.20847428205128204|
|2018|          Les_Corts|0.47042419298245614|
|2017|        Sant_Andreu|0.21254642105263158|
|2016|             Gràcia| 0.2653363563218391|
|2016|         Sant_Martí|0.21905444217687076|
|2015|           Eixample|0.29212261849710985|
|2019|           Eixample| 0.3578624161849711|
|2018|         Nou_Barris|0.14967166666666668|
|2017|     Horta-Guinardó|0.20365434146341463|
|2016|Sarrià-Sant_Gervasi| 0.5873470102040816|
|2021|         Nou_Barris|0.17628994871794873|
|2020|         Nou_Barris| 0.1632850170940171|
|2019|         Nou_Barris| 0.1655876923076923|
|2018|           Eixample|0.33227485549132946|
|2017|Sarrià-Sant_Gervasi| 0.6134964693877552|
|2016|     Horta-Guinardó|0.19813920325203252|
|2015|       Ciutat_Vella|0.08592322222222222|
|2015|     Sa

In [28]:
# Crear una columna 'Nom_Districte' en DF3 que no conté accents ni la L
treure_accents_L_udf = udf(treure_accents_L, StringType())

DF3 = DF3.withColumn('Nom_Districte', treure_accents_L_udf(DF3['Nom_Districte']))

# Mostrem el nou DF3
DF3.show()

+----+-------------------+-------------------+
| Any|      Nom_Districte|    District_Income|
+----+-------------------+-------------------+
|2021|       Ciutat_Vella|0.13931579629629628|
|2021|     Sants-Montjuic|0.24206200854700854|
|2021|          Les_Corts| 0.4859704561403509|
|2020|       Ciutat_Vella|0.12283433333333334|
|2020|     Sants-Montjuic|0.22588576923076922|
|2020|          Les_Corts|0.47797108771929825|
|2019|       Ciutat_Vella|0.13657272222222222|
|2019|     Sants-Montjuic| 0.2292812393162393|
|2019|          Les_Corts| 0.4899912280701754|
|2017|             Gracia| 0.2726322298850575|
|2017|         Sant_Marti| 0.2257753673469388|
|2016|        Sant_Andreu|0.20619405263157894|
|2015|         Nou_Barris|0.12353695726495727|
|2019|           Eixample| 0.3578624161849711|
|2018|         Nou_Barris|0.14967166666666668|
|2017|     Horta-Guinardo|0.20365434146341463|
|2016|Sarria-Sant_Gervasi| 0.5873470102040816|
|2021|Sarria-Sant_Gervasi| 0.6646924591836735|
|2020|Sarria-

In [29]:
# Renombrar columnes
DF3 = DF3.withColumnRenamed("Nom_Districte", "District")
DF3 = DF3.withColumnRenamed("Any", "Year")

In [30]:
# Utilitzem la funció createOrReplaceTempView per a poder fer consultes SQL de manera més familiar
DF1.createOrReplaceTempView("taula1")
DF2.createOrReplaceTempView("taula2")
DF3.createOrReplaceTempView("taula3")

In [32]:
# En aquesta query fem un join de les tres taules per a obtenir la informació que volem entre Year i district
DF = spark.sql("""
    SELECT t1.Year,t1.District, t1.Year_Price_month, t2.Year_Price_m2, t3.District_Income
    FROM taula1 t1, taula2 t2, taula3 t3
    WHERE t1.District = t2.District AND t1.District = t3.District and t1.Year = t2.Year and t1.Year = t3.Year
""")
DF.show()

+----+-------------------+-------------------+-------------------+-------------------+
|Year|           District|   Year_Price_month|      Year_Price_m2|    District_Income|
+----+-------------------+-------------------+-------------------+-------------------+
|2015|     Horta-Guinardo|0.23929853325366826|0.36386151940989603|0.18996447967479674|
|2021|          Les_Corts| 0.5665535385146733| 0.5975901398086828| 0.4859704561403509|
|2016|     Sants-Montjuic| 0.2749316692108045| 0.4447362411509477| 0.1827870170940171|
|2015|     Sants-Montjuic|  0.247910301005466|0.38754598969830756| 0.1763603247863248|
|2019|     Sants-Montjuic|0.36016970316529145| 0.5404201872573647| 0.2292812393162393|
|2021|Sarria-Sant_Gervasi|  0.596762896080691| 0.6530077262693158| 0.6646924591836735|
|2020|          Les_Corts| 0.5927254369178394| 0.6444996320824136|0.47797108771929825|
|2016|         Nou_Barris|  0.204482600232365|0.32688251165072363| 0.1293284017094017|
|2020|        Sant_Andreu| 0.33025385111489

In [33]:
# Sobreescrivim la taula nova_base amb la nova informació
DF.write \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:exploited_zone/freshdata_exploited1.db") \
        .option("dbtable", "nova_base") \
        .option("driver", "org.duckdb.DuckDBDriver") \
        .mode("overwrite") \
        .save()

CodeCache: size=131072Kb used=28158Kb max_used=28158Kb free=102913Kb
 bounds [0x00000001091e0000, 0x000000010ad90000, 0x00000001111e0000]
 total_blobs=11029 nmethods=10085 adapters=855
 compilation: disabled (not enough contiguous free space left)


Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


### Preparació per al segon pipeline (Anàlisi descriptiu)

In [34]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

In [35]:
# En aquesta query necessitem només les columnes seleccionades per realitzar l'anàlisi descriptiu
DF1 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:trusted_zone/freshdata_trusted.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", '''SELECT b.Any, b.Trimestre, b.Nom_Districte, b.Nom_Barri, b."Superfície_mitjana_(m2_construïts)", b.Nombre  FROM compravenda_sup b''') \
  .load()
DF1.show()

+----+---------+-------------+--------------------+----------------------------------+------------------+
| Any|Trimestre|Nom_Districte|           Nom_Barri|Superfície_mitjana_(m2_construïts)|            Nombre|
+----+---------+-------------+--------------------+----------------------------------+------------------+
|2023|        1|       Gràcia|   la_Vila_de_Gràcia|                             Total|              80.5|
|2023|        2|       Gràcia|   la_Vila_de_Gràcia|                             Total| 81.30000305175781|
|2023|        3|       Gràcia|   la_Vila_de_Gràcia|                             Total|  83.0999984741211|
|2023|        1|       Gràcia|el_Camp_d_en_Gras...|                             Total| 77.30000305175781|
|2023|        2|       Gràcia|el_Camp_d_en_Gras...|                             Total|  71.4000015258789|
|2023|        3|       Gràcia|el_Camp_d_en_Gras...|                             Total|              68.5|
|2023|        1|       Gràcia|             el_

In [36]:
# Sobreescriure la taula nova_base amb la nova informació
DF1.write \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:exploited_zone/freshdata_exploited2.db") \
        .option("dbtable", "nova_base") \
        .option("driver", "org.duckdb.DuckDBDriver") \
        .mode("overwrite") \
        .save()